In [1404]:
import re
import string
import pythainlp
from pythainlp import word_tokenize
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
import nltk
p_stemmer = PorterStemmer()

def split_word(text):
            
    
    tokens = word_tokenize(text,engine='newmm')
    
  
    
    # หารากศัพท์ภาษาไทย และภาษาอังกฤษ
    # English
    tokens = [p_stemmer.stem(i) for i in tokens]
    
    # Thai
    tokens_temp=[]
    for i in tokens:
        w_syn = wordnet.synsets(i)
        if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
            tokens_temp.append(w_syn[0].lemma_names('tha')[0])
        else:
            tokens_temp.append(i)
    
    tokens = tokens_temp
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

import xlrd  
import pandas as pd
Corpus = pd.read_excel('corpus_comments_thai2fit.xlsx')
#Corpus['comments'] = Corpus['comments'].astype('str')
Corpus['reaction_num'] = Corpus['reaction_num'].astype('float')
Corpus['reply_num'] = Corpus['reply_num'].astype('float')
Corpus['reaction1'] = Corpus['reaction1'].astype('str')
Corpus['reaction2'] = Corpus['reaction2'].astype('str')
Corpus['reaction3'] = Corpus['reaction3'].astype('str')
#Corpus['d_1'] = Corpus['d_1'].astype('float')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Train_X, Test_X, Train_Y, Test_Y = train_test_split(Corpus[['d_0','d_1','d_2','d_3','d_4','d_5','d_6','d_7','d_8','d_9','d_10','d_11','d_12','d_13','d_14','d_15',
                                                            'd_16','d_17','d_18','d_19','d_20','d_21','d_22','d_23','d_24','d_25','d_26','d_27','d_28','d_29','d_30','d_31','d_32','d_33','d_34','d_35','d_36','d_37','d_38','d_39','d_40','d_41','d_42','d_43','d_44','d_45','d_46','d_47','d_48','d_49','d_50','d_51','d_52','d_53','d_54','d_55','d_56','d_57','d_58','d_59','d_60','d_61','d_62','d_63','d_64','d_65','d_66','d_67','d_68','d_69','d_70','d_71','d_72','d_73','d_74','d_75','d_76','d_77','d_78','d_79','d_80','d_81','d_82','d_83','d_84','d_85','d_86','d_87','d_88','d_89','d_90','d_91','d_92','d_93','d_94','d_95','d_96','d_97','d_98','d_99',
                                                            'reaction_num','reply_num','reaction1','reaction2','reaction3']], Corpus[['Rude','Figurative','Offensive','Dirty']], test_size=0.2,random_state=1000);


In [1405]:
from sklearn.metrics import classification_report
from  sklearn.metrics import precision_recall_fscore_support
def pandas_classification_report(y_true, y_pred):
    metrics_summary = precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred)

    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='weighted'))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index)

    support = class_report_df.loc['support']
    
#     total = support.sum() 
#     avg[-1] = total

#     class_report_df['avg / total'] = avg

    return class_report_df.T

In [1406]:
import csv
import itertools
def writeEvaluation(s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16):
    f = open("classification_report_thai2fit.csv", mode='a')
    f.write(s1+","+s2+","+s3+","+s4)
    f.write("\n")
    f.write(s5+","+s6+","+s7+","+s8)
    f.write("\n")
    f.write(s9+","+s10+","+s11+","+s12)
    f.write("\n")
    f.write(s13+","+s14+","+s15+","+s16)
    f.write("\n")
    f.close()
    
def writeEvaluation_threshold_1_minus_threshold(s1,s2,s3,s4,s5,s6,s7,s8):
    f = open("classification_report_thai2fit.csv", mode='a')
    f.write(s1+","+s2+","+s3+","+s4)
    f.write("\n")
    f.write(s5+","+s6+","+s7+","+s8)
    f.write("\n")
    f.close()

In [1407]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
string='';
def report_performance(predictions_class_train, predictions_class_test, predictions_class_train_c, predictions_class_test_c, type_class,threshold):
    #Evaluation class (test data) 
    str1 = "Test data (ensemble)"
    str2 = str(threshold)
    str3 = str(type_class)
    str4 = str(accuracy_score(predictions_class_test, Test_Y[type_class]))
    print(type_class+" Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature): ",
          accuracy_score(predictions_class_test, Test_Y[type_class])*100)
    print("Confusion Matrix: \n", confusion_matrix(Test_Y[type_class],predictions_class_test))
    report = classification_report(Test_Y[type_class],predictions_class_test)
    print("PRF: \n", report)
    
    print("-----------------------------------------------------------\n")
    str5 = "Test data (multiple features)"
    str6 = str(threshold)
    str7 = str(type_class)
    str8 = str(accuracy_score(predictions_class_test_c, Test_Y[type_class]))
    print(type_class+" Accuracy score(test data with multiple features):", 
          accuracy_score(predictions_class_test_c, Test_Y[type_class])*100)
    print("Confusion Matrix: \n", confusion_matrix(Test_Y[type_class],predictions_class_test_c))
    report = classification_report(Test_Y[type_class],predictions_class_test_c);
    print("PRF: \n", report)

    print("-----------------------------------------------------------\n")

    #Evaluation class (train data)
    str9 = "Train data (ensemble)"
    str10 = str(threshold)
    str11 = str(type_class)
    str12 = str(accuracy_score(predictions_class_train, Train_Y[type_class]))
    print(type_class+" Accuracy score(train data with ensemble (1 model: content feature, 1 model: context feature): ",
          accuracy_score(predictions_class_train, Train_Y[type_class])*100)
    print("Confusion Matrix: \n",confusion_matrix(Train_Y[type_class],predictions_class_train))
    print("PRF: \n",classification_report(Train_Y[type_class],predictions_class_train))
    print("-----------------------------------------------------------\n")
    str13 = "Train data (mulitple features)"
    str14 = str(threshold)
    str15 = str(type_class)
    str16 = str(accuracy_score(predictions_class_train_c, Train_Y[type_class]))
    print(type_class+" Accuracy score(train data with multiple features):", 
          accuracy_score(predictions_class_train_c, Train_Y[type_class])*100)
    print("Confusion Matrix: \n",confusion_matrix(Train_Y[type_class],predictions_class_train_c))
    print("PRF: \n",classification_report(Train_Y[type_class],predictions_class_train_c))
    print("___________________________________________________________________________________________________________\n")

    writeEvaluation(str1,str2,str3,str4,str5,str6,str7,str8,str9,str10,str11,str12,str13,str14,str15,str16)


In [1408]:
string='';
def report_performance_threshold_1_minus_threshold(predictions_class_train, predictions_class_test, type_class,threshold):
     #Evaluation class (test data) 
    str1 = "Test data ensemble(content context (threshold & 1-threshold))"
    str2 = str(threshold)
    str3 = str(type_class)
    str4 = str(accuracy_score(predictions_class_test, Test_Y[type_class]))
    print(type_class+" Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature): ",
          accuracy_score(predictions_class_test, Test_Y[type_class])*100)
    print("Confusion Matrix: \n", confusion_matrix(Test_Y[type_class],predictions_class_test))
    report = classification_report(Test_Y[type_class],predictions_class_test)
    print("PRF: \n", report)
    print("-----------------------------------------------------------\n")
    
    #Evaluation class (train data)
    str5 = "Train data ensemble(content context (threshold & 1-threshold)"
    str6 = str(threshold)
    str7 = str(type_class)
    str8 = str(accuracy_score(predictions_class_train, Train_Y[type_class]))
    print(type_class+" Accuracy score(train data with ensemble (1 model: content feature, 1 model: context feature): ",
          accuracy_score(predictions_class_train, Train_Y[type_class])*100)
    print("Confusion Matrix: \n",confusion_matrix(Train_Y[type_class],predictions_class_train))
    print("PRF: \n",classification_report(Train_Y[type_class],predictions_class_train))
    print("___________________________________________________________________________________________________________\n")

    writeEvaluation_threshold_1_minus_threshold(str1,str2,str3,str4,str5,str6,str7,str8)


In [1409]:
def predict_class(list_predict_prob,threshold):
    list_predict_class = [];
    for i in range (len(list_predict_prob)):
        if(list_predict_prob[i][1] > threshold):
            list_predict_class.append(1)
        else :
            list_predict_class.append(0)
    return list_predict_class

In [1410]:
def changeStrInReactionToNum(reaction_data=[]):
    reaction = []
    for i in reaction_data:
        if(i == 'haha'):
            reaction.append(1)
        elif(i == 'like'):
            reaction.append(2)
        elif(i == 'angry'):
            reaction.append(3)
        elif(i == 'sad'):
            reaction.append(4)
        elif(i == 'wow'):
            reaction.append(5)
        elif(i == 'love'):
            reaction.append(6)
        else:
            reaction.append(7)
    return reaction

In [1411]:
from scipy.sparse import csr_matrix
from statistics import mean 
#content feature
train_content = [];
test_content = [];
for i in range(0,100):
    #need to determine 10 decimals place because float 64 does not support large value
    decimals = 10   
    #minimum value of Train_X['d_'+str(i)] need to be added due to in fit function, input X must be non-negative
    train_x_min = abs(round(Train_X['d_'+str(i)],decimals).min())
    test_x_min = abs(round(Test_X['d_'+str(i)],decimals).min())
    
    Train_X['d_'+str(i)] = round(Train_X['d_'+str(i)],decimals)+train_x_min
    Test_X['d_'+str(i)] = round(Test_X['d_'+str(i)],decimals)+test_x_min
    #replace nan value with mean value of Train_X['d_'+str(i)]
    Train_X['d_'+str(i)] = Train_X['d_'+str(i)].replace(np.nan, np.nanmean(Train_X['d_'+str(i)]))
    Test_X['d_'+str(i)] = Test_X['d_'+str(i)].replace(np.nan, np.nanmean(Test_X['d_'+str(i)])) 
    train_content.append(csr_matrix(Train_X['d_'+str(i)].values.reshape(-1,1)))
    test_content.append(csr_matrix(Test_X['d_'+str(i)].values.reshape(-1,1)))

for w in range(0,100):
    for j in train_content[w]:
        if(j<0):
            print(j)
import numpy as np
#context feature
train_reaction_num = csr_matrix(Train_X['reaction_num'].values.reshape(-1, 1))
test_reaction_num = csr_matrix(Test_X['reaction_num'].values.reshape(-1, 1))
train_reply_num = csr_matrix(Train_X['reply_num'].values.reshape(-1,1))
test_reply_num = csr_matrix(Test_X['reply_num'].values.reshape(-1,1))

# Train_X['reaction1'] = np.nan_to_num(0)
# Test_X['reaction1'] = np.nan_to_num(0)
# Train_X['reaction2'] = np.nan_to_num(0)
# Test_X['reaction2'] = np.nan_to_num(0)
# Train_X['reaction3'] = np.nan_to_num(0)
# Test_X['reaction3'] = np.nan_to_num(0)
# train_reaction1_feat = csr_matrix(Train_X['reaction1'].values.reshape(-1,1))
# test_reaction1_feat = csr_matrix(Test_X['reaction1'].values.reshape(-1, 1))
# train_reaction2_feat = csr_matrix(Train_X['reaction2'].values.reshape(-1, 1))
# test_reaction2_feat = csr_matrix(Test_X['reaction2'].values.reshape(-1, 1))
# train_reaction3_feat = csr_matrix(Train_X['reaction3'].values.reshape(-1, 1))
# test_reaction3_feat = csr_matrix(Test_X['reaction3'].values.reshape(-1, 1))

train_reaction1 = changeStrInReactionToNum(Train_X['reaction1'])
test_reaction1 = changeStrInReactionToNum(Test_X['reaction1'])
train_reaction2 = changeStrInReactionToNum(Train_X['reaction2'])
test_reaction2 =  changeStrInReactionToNum(Test_X['reaction2'])
train_reaction3 = changeStrInReactionToNum(Train_X['reaction3'])
test_reaction3=  changeStrInReactionToNum(Test_X['reaction3'])

train_reaction1_feat = csr_matrix(train_reaction1).reshape(-1,1)
test_reaction1_feat = csr_matrix(test_reaction1).reshape(-1,1)
train_reaction2_feat = csr_matrix(train_reaction2).reshape(-1,1)
test_reaction2_feat = csr_matrix(test_reaction2).reshape(-1,1)
train_reaction3_feat = csr_matrix(train_reaction3).reshape(-1,1)
test_reaction3_feat = csr_matrix(test_reaction3).reshape(-1,1)


In [1412]:
from scipy.sparse import hstack, csr_matrix
train_feat = hstack([train_content[0],train_content[1],train_content[2],train_content[3],train_content[4],train_content[5],train_content[6],train_content[7],train_content[8],train_content[9],train_content[10],train_content[11],train_content[12],train_content[13],train_content[14],train_content[15],train_content[16],train_content[17],train_content[18],train_content[19],train_content[20],train_content[21],train_content[22],train_content[23],train_content[24],train_content[25],train_content[26],train_content[27],train_content[28],train_content[29],train_content[30],train_content[31],train_content[32],train_content[33],train_content[34],train_content[35],train_content[36],train_content[37],train_content[38],train_content[39],train_content[40],train_content[41],train_content[42],train_content[43],train_content[44],train_content[45],train_content[46],train_content[47],train_content[48],train_content[49],train_content[50],train_content[51],train_content[52],train_content[53],train_content[54],train_content[55],train_content[56],train_content[57],train_content[58],train_content[59],train_content[60],train_content[61],train_content[62],train_content[63],train_content[64],train_content[65],train_content[66],train_content[67],train_content[68],train_content[69],train_content[70],train_content[71],train_content[72],train_content[73],train_content[74],train_content[75],train_content[76],train_content[77],train_content[78],train_content[79],train_content[80],train_content[81],train_content[82],train_content[83],train_content[84],train_content[85],train_content[86],train_content[87],train_content[88],train_content[89],train_content[90],train_content[91],train_content[92],train_content[93],train_content[94],train_content[95],train_content[96],train_content[97],train_content[98],train_content[99], train_reaction_num, train_reply_num, train_reaction1_feat, train_reaction2_feat, train_reaction3_feat])
test_feat = hstack([test_content[0],test_content[1],test_content[2],test_content[3],test_content[4],test_content[5],test_content[6],test_content[7],test_content[8],test_content[9],test_content[10],test_content[11],test_content[12],test_content[13],test_content[14],test_content[15],test_content[16],test_content[17],test_content[18],test_content[19],test_content[20],test_content[21],test_content[22],test_content[23],test_content[24],test_content[25],test_content[26],test_content[27],test_content[28],test_content[29],test_content[30],test_content[31],test_content[32],test_content[33],test_content[34],test_content[35],test_content[36],test_content[37],test_content[38],test_content[39],test_content[40],test_content[41],test_content[42],test_content[43],test_content[44],test_content[45],test_content[46],test_content[47],test_content[48],test_content[49],test_content[50],test_content[51],test_content[52],test_content[53],test_content[54],test_content[55],test_content[56],test_content[57],test_content[58],test_content[59],test_content[60],test_content[61],test_content[62],test_content[63],test_content[64],test_content[65],test_content[66],test_content[67],test_content[68],test_content[69],test_content[70],test_content[71],test_content[72],test_content[73],test_content[74],test_content[75],test_content[76],test_content[77],test_content[78],test_content[79],test_content[80],test_content[81],test_content[82],test_content[83],test_content[84],test_content[85],test_content[86],test_content[87],test_content[88],test_content[89],test_content[90],test_content[91],test_content[92],test_content[93],test_content[94],test_content[95],test_content[96],test_content[97],test_content[98],test_content[99],test_reaction_num, test_reply_num, test_reaction1_feat, test_reaction2_feat, test_reaction3_feat])
train_feat_content = hstack([train_content[0],train_content[1],train_content[2],train_content[3],train_content[4],train_content[5],train_content[6],train_content[7],train_content[8],train_content[9],train_content[10],train_content[11],train_content[12],train_content[13],train_content[14],train_content[15],train_content[16],train_content[17],train_content[18],train_content[19],train_content[20],train_content[21],train_content[22],train_content[23],train_content[24],train_content[25],train_content[26],train_content[27],train_content[28],train_content[29],train_content[30],train_content[31],train_content[32],train_content[33],train_content[34],train_content[35],train_content[36],train_content[37],train_content[38],train_content[39],train_content[40],train_content[41],train_content[42],train_content[43],train_content[44],train_content[45],train_content[46],train_content[47],train_content[48],train_content[49],train_content[50],train_content[51],train_content[52],train_content[53],train_content[54],train_content[55],train_content[56],train_content[57],train_content[58],train_content[59],train_content[60],train_content[61],train_content[62],train_content[63],train_content[64],train_content[65],train_content[66],train_content[67],train_content[68],train_content[69],train_content[70],train_content[71],train_content[72],train_content[73],train_content[74],train_content[75],train_content[76],train_content[77],train_content[78],train_content[79],train_content[80],train_content[81],train_content[82],train_content[83],train_content[84],train_content[85],train_content[86],train_content[87],train_content[88],train_content[89],train_content[90],train_content[91],train_content[92],train_content[93],train_content[94],train_content[95],train_content[96],train_content[97],train_content[98],train_content[99]])
test_feat_content = hstack([test_content[0],test_content[1],test_content[2],test_content[3],test_content[4],test_content[5],test_content[6],test_content[7],test_content[8],test_content[9],test_content[10],test_content[11],test_content[12],test_content[13],test_content[14],test_content[15],test_content[16],test_content[17],test_content[18],test_content[19],test_content[20],test_content[21],test_content[22],test_content[23],test_content[24],test_content[25],test_content[26],test_content[27],test_content[28],test_content[29],test_content[30],test_content[31],test_content[32],test_content[33],test_content[34],test_content[35],test_content[36],test_content[37],test_content[38],test_content[39],test_content[40],test_content[41],test_content[42],test_content[43],test_content[44],test_content[45],test_content[46],test_content[47],test_content[48],test_content[49],test_content[50],test_content[51],test_content[52],test_content[53],test_content[54],test_content[55],test_content[56],test_content[57],test_content[58],test_content[59],test_content[60],test_content[61],test_content[62],test_content[63],test_content[64],test_content[65],test_content[66],test_content[67],test_content[68],test_content[69],test_content[70],test_content[71],test_content[72],test_content[73],test_content[74],test_content[75],test_content[76],test_content[77],test_content[78],test_content[79],test_content[80],test_content[81],test_content[82],test_content[83],test_content[84],test_content[85],test_content[86],test_content[87],test_content[88],test_content[89],test_content[90],test_content[91],test_content[92],test_content[93],test_content[94],test_content[95],test_content[96],test_content[97],test_content[98],test_content[99]])
train_feat_context = hstack([train_reaction_num, train_reply_num, train_reaction1_feat, train_reaction2_feat, train_reaction3_feat])
test_feat_context = hstack([test_reaction_num, test_reply_num, test_reaction1_feat, test_reaction2_feat, test_reaction3_feat])

In [1413]:
#Naive bays
import numpy
from sklearn.naive_bayes import MultinomialNB
#ensemble model of comments feature and context feature
abusive_type = ["Rude","Figurative","Offensive","Dirty"]
report=[]
for threshold in numpy.arange(0.1,1.1,0.1):
    for i in range(0,4):
        Naive_bayes_1 = MultinomialNB()
        Naive_bayes_2 = MultinomialNB()

        Naive_bayes_1.fit(train_feat_content, Train_Y[abusive_type[i]])
        Naive_bayes_2.fit(train_feat_context, Train_Y[abusive_type[i]])

        predictions_NB_Train1 = Naive_bayes_1.predict_proba(train_feat_content)
        predictions_NB_Train2 = Naive_bayes_2.predict_proba(train_feat_context)

        predictions_NB_Test1 = Naive_bayes_1.predict_proba(test_feat_content)
        predictions_NB_Test2 = Naive_bayes_2.predict_proba(test_feat_context)

        predictions_NB_Train = (predictions_NB_Train1+predictions_NB_Train2)/2
        predictions_NB_Test = (predictions_NB_Test1+predictions_NB_Test2)/2

        predictions_class_Train = predict_class(predictions_NB_Train,threshold)
        predictions_class_Test = predict_class(predictions_NB_Test,threshold)

        #combine multiple feature with one model
        Naive_bayes_3 = MultinomialNB()
        Naive_bayes_3.fit(train_feat, Train_Y[abusive_type[i]])
        predictions_NB_Train3 = Naive_bayes_3.predict_proba(train_feat)
        predictions_NB_Test3 = Naive_bayes_3.predict_proba(test_feat)

        predictions_class_Train_c = (predictions_NB_Train3 [:,1] >= threshold).astype('int')
        predictions_class_Test_c = (predictions_NB_Test3 [:,1] >= threshold).astype('int')
        report_performance(predictions_class_Train, predictions_class_Test, predictions_class_Train_c,predictions_class_Test_c, abusive_type[i],threshold)

Rude Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature):  62.33766233766234
Confusion Matrix: 
 [[  0 203]
 [  0 336]]
PRF: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       203
           1       0.62      1.00      0.77       336

    accuracy                           0.62       539
   macro avg       0.31      0.50      0.38       539
weighted avg       0.39      0.62      0.48       539

-----------------------------------------------------------

Rude Accuracy score(test data with multiple features): 63.26530612244898
Confusion Matrix: 
 [[ 29 174]
 [ 24 312]]
PRF: 
               precision    recall  f1-score   support

           0       0.55      0.14      0.23       203
           1       0.64      0.93      0.76       336

    accuracy                           0.63       539
   macro avg       0.59      0.54      0.49       539
weighted avg       0.61      0.63      0.56       

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


PRF: 
               precision    recall  f1-score   support

           0       0.96      0.91      0.93      2052
           1       0.09      0.17      0.12       103

    accuracy                           0.88      2155
   macro avg       0.52      0.54      0.53      2155
weighted avg       0.92      0.88      0.89      2155

___________________________________________________________________________________________________________

Offensive Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature):  26.34508348794063
Confusion Matrix: 
 [[118 390]
 [  7  24]]
PRF: 
               precision    recall  f1-score   support

           0       0.94      0.23      0.37       508
           1       0.06      0.77      0.11        31

    accuracy                           0.26       539
   macro avg       0.50      0.50      0.24       539
weighted avg       0.89      0.26      0.36       539

----------------------------------------------------------

PRF: 
               precision    recall  f1-score   support

           0       0.99      0.92      0.96       536
           1       0.00      0.00      0.00         3

    accuracy                           0.91       539
   macro avg       0.50      0.46      0.48       539
weighted avg       0.99      0.91      0.95       539

-----------------------------------------------------------

Dirty Accuracy score(test data with multiple features): 89.6103896103896
Confusion Matrix: 
 [[483  53]
 [  3   0]]
PRF: 
               precision    recall  f1-score   support

           0       0.99      0.90      0.95       536
           1       0.00      0.00      0.00         3

    accuracy                           0.90       539
   macro avg       0.50      0.45      0.47       539
weighted avg       0.99      0.90      0.94       539

-----------------------------------------------------------

Dirty Accuracy score(train data with ensemble (1 model: content feature, 1 model: context feat

Figurative Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature):  86.08534322820037
Confusion Matrix: 
 [[462  46]
 [ 29   2]]
PRF: 
               precision    recall  f1-score   support

           0       0.94      0.91      0.92       508
           1       0.04      0.06      0.05        31

    accuracy                           0.86       539
   macro avg       0.49      0.49      0.49       539
weighted avg       0.89      0.86      0.87       539

-----------------------------------------------------------

Figurative Accuracy score(test data with multiple features): 83.85899814471243
Confusion Matrix: 
 [[450  58]
 [ 29   2]]
PRF: 
               precision    recall  f1-score   support

           0       0.94      0.89      0.91       508
           1       0.03      0.06      0.04        31

    accuracy                           0.84       539
   macro avg       0.49      0.48      0.48       539
weighted avg       0.89      0.84     

...

Rude Accuracy score(train data with multiple features): 64.45475638051043
Confusion Matrix: 
 [[ 346  519]
 [ 247 1043]]
PRF: 
               precision    recall  f1-score   support

           0       0.58      0.40      0.47       865
           1       0.67      0.81      0.73      1290

    accuracy                           0.64      2155
   macro avg       0.63      0.60      0.60      2155
weighted avg       0.63      0.64      0.63      2155

___________________________________________________________________________________________________________

Figurative Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature):  94.24860853432281
Confusion Matrix: 
 [[508   0]
 [ 31   0]]
PRF: 
               precision    recall  f1-score   support

           0       0.94      1.00      0.97       508
           1       0.00      0.00      0.00        31

    accuracy                           0.94       539
   macro avg       0.47      0.50      0.49  

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 

Offensive Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature):  94.24860853432281
Confusion Matrix: 
 [[508   0]
 [ 31   0]]
PRF: 
               precision    recall  f1-score   support

           0       0.94      1.00      0.97       508
           1       0.00      0.00      0.00        31

    accuracy                           0.94       539
   macro avg       0.47      0.50      0.49       539
weighted avg       0.89      0.94      0.91       539

-----------------------------------------------------------

Offensive Accuracy score(test data with multiple features): 55.84415584415584
Confusion Matrix: 
 [[283 225]
 [ 13  18]]
PRF: 
               precision    recall  f1-score   support

           0       0.96      0.56      0.70       508
           1       0.07      0.58      0.13        31

    accuracy                           0.56       539
   macro avg       0.52      0.57      0.42       539
weighted avg       0.91      0.56      0

PRF: 
               precision    recall  f1-score   support

           0       0.99      0.97      0.98      2126
           1       0.02      0.03      0.02        29

    accuracy                           0.96      2155
   macro avg       0.50      0.50      0.50      2155
weighted avg       0.97      0.96      0.97      2155

___________________________________________________________________________________________________________



/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [1414]:
#Naive bays
import numpy
from sklearn.naive_bayes import MultinomialNB
#P=threshold*(content)+(1-threshold)*(context)
#ensemble model of comments feature and context feature
abusive_type = ["Rude","Figurative","Offensive","Dirty"]
report=[]
for threshold in numpy.arange(0.1,1.1,0.1):
    for i in range(0,4):
        Naive_bayes_1 = MultinomialNB()
        Naive_bayes_2 = MultinomialNB()

        Naive_bayes_1.fit(train_feat_content, Train_Y[abusive_type[i]])
        Naive_bayes_2.fit(train_feat_context, Train_Y[abusive_type[i]])

        predictions_NB_Train1 = Naive_bayes_1.predict_proba(train_feat_content)
        predictions_NB_Train2 = Naive_bayes_2.predict_proba(train_feat_context)

        predictions_NB_Test1 = Naive_bayes_1.predict_proba(test_feat_content)
        predictions_NB_Test2 = Naive_bayes_2.predict_proba(test_feat_context)

        predictions_NB_Train = (predictions_NB_Train1*threshold)+(predictions_NB_Train2*(1-threshold))
        predictions_NB_Test = (predictions_NB_Test1*threshold)+(predictions_NB_Test2*(1-threshold))

        predictions_class_Train = predict_class(predictions_NB_Train,threshold)
        predictions_class_Test = predict_class(predictions_NB_Test,threshold)

        report_performance_threshold_1_minus_threshold(predictions_class_Train, predictions_class_Test, abusive_type[i],threshold)

Rude Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature):  63.4508348794063
Confusion Matrix: 
 [[ 25 178]
 [ 19 317]]
PRF: 
               precision    recall  f1-score   support

           0       0.57      0.12      0.20       203
           1       0.64      0.94      0.76       336

    accuracy                           0.63       539
   macro avg       0.60      0.53      0.48       539
weighted avg       0.61      0.63      0.55       539

-----------------------------------------------------------

Rude Accuracy score(train data with ensemble (1 model: content feature, 1 model: context feature):  59.62877030162414
Confusion Matrix: 
 [[  70  795]
 [  75 1215]]
PRF: 
               precision    recall  f1-score   support

           0       0.48      0.08      0.14       865
           1       0.60      0.94      0.74      1290

    accuracy                           0.60      2155
   macro avg       0.54      0.51      0.44      2155
we

Offensive Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature):  26.53061224489796
Confusion Matrix: 
 [[119 389]
 [  7  24]]
PRF: 
               precision    recall  f1-score   support

           0       0.94      0.23      0.38       508
           1       0.06      0.77      0.11        31

    accuracy                           0.27       539
   macro avg       0.50      0.50      0.24       539
weighted avg       0.89      0.27      0.36       539

-----------------------------------------------------------

Offensive Accuracy score(train data with ensemble (1 model: content feature, 1 model: context feature):  26.77494199535963
Confusion Matrix: 
 [[ 467 1553]
 [  25  110]]
PRF: 
               precision    recall  f1-score   support

           0       0.95      0.23      0.37      2020
           1       0.07      0.81      0.12       135

    accuracy                           0.27      2155
   macro avg       0.51      0.52      0.25  

PRF: 
               precision    recall  f1-score   support

           0       0.64      0.47      0.54       203
           1       0.72      0.84      0.78       336

    accuracy                           0.70       539
   macro avg       0.68      0.66      0.66       539
weighted avg       0.69      0.70      0.69       539

-----------------------------------------------------------

Rude Accuracy score(train data with ensemble (1 model: content feature, 1 model: context feature):  66.5429234338747
Confusion Matrix: 
 [[ 319  546]
 [ 175 1115]]
PRF: 
               precision    recall  f1-score   support

           0       0.65      0.37      0.47       865
           1       0.67      0.86      0.76      1290

    accuracy                           0.67      2155
   macro avg       0.66      0.62      0.61      2155
weighted avg       0.66      0.67      0.64      2155

___________________________________________________________________________________________________________

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 

Dirty Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature):  99.44341372912801
Confusion Matrix: 
 [[536   0]
 [  3   0]]
PRF: 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00       536
           1       0.00      0.00      0.00         3

    accuracy                           0.99       539
   macro avg       0.50      0.50      0.50       539
weighted avg       0.99      0.99      0.99       539

-----------------------------------------------------------

Dirty Accuracy score(train data with ensemble (1 model: content feature, 1 model: context feature):  98.65429234338747
Confusion Matrix: 
 [[2126    0]
 [  29    0]]
PRF: 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      2126
           1       0.00      0.00      0.00        29

    accuracy                           0.99      2155
   macro avg       0.49      0.50      0.50      2155

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 